In [8]:
import pandas as pd

df = pd.read_csv('Score_Results.csv')

df

,epochs,batch_size,learning_rate,lambda_weight,loss_function,ari,hvg,asw,f1,nmi,sil,graph,pcr,sil_batch,avg_bio,avg_batch,time
0,30,640,0.00100,0.8,uniform,0.331539,0.13875,0.631585,0.914867,0.660978,0.608712,0.907090,0.496466,0.725905,0.547738,0.709820,570.702047
1,30,640,0.00010,0.8,uniform,0.331539,0.26275,0.746295,0.981993,0.660978,0.691858,0.848513,0.421729,0.847236,0.612569,0.705826,457.356091
2,30,640,0.00001,0.8,uniform,0.331539,0.32800,0.697628,0.872914,0.660978,0.619759,0.818117,0.451094,0.852583,0.585136,0.707265,461.706384
